# Load Ni-Mo data

In [1]:
from pymatgen.core import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]
train_stresses = [d['outputs']['stress'] for d in data]

# Set up the MTP and train

In [2]:
from maml.apps.pes import MTPotential

mtp = MTPotential()

In [3]:
mtp.train(train_structures=train_structures, train_energies=train_energies,
          train_forces=train_forces, train_stresses = None, max_dist=5, stress_weight=0)

INFO:maml.utils._lammps:Structure index 0 is rotated.
INFO:maml.utils._lammps:Structure index 1 is rotated.
INFO:maml.utils._lammps:Structure index 2 is rotated.
INFO:maml.utils._lammps:Structure index 3 is rotated.
INFO:maml.utils._lammps:Structure index 4 is rotated.
INFO:maml.utils._lammps:Structure index 5 is rotated.
INFO:maml.utils._lammps:Structure index 6 is rotated.
INFO:maml.utils._lammps:Structure index 7 is rotated.
INFO:maml.utils._lammps:Structure index 8 is rotated.
INFO:maml.utils._lammps:Structure index 9 is rotated.


0

# Predict and evaluate the energies and forces of training data

In [3]:
df_orig, df_predict = mtp.evaluate(test_structures=train_structures, 
                                   test_energies=train_energies,
                                   test_forces=train_forces, test_stresses=train_stresses)

INFO:maml.utils._lammps:Structure index 0 is rotated.
INFO:maml.utils._lammps:Structure index 1 is rotated.
INFO:maml.utils._lammps:Structure index 2 is rotated.
INFO:maml.utils._lammps:Structure index 3 is rotated.
INFO:maml.utils._lammps:Structure index 4 is rotated.
INFO:maml.utils._lammps:Structure index 5 is rotated.
INFO:maml.utils._lammps:Structure index 6 is rotated.
INFO:maml.utils._lammps:Structure index 7 is rotated.
INFO:maml.utils._lammps:Structure index 8 is rotated.
INFO:maml.utils._lammps:Structure index 9 is rotated.


In [4]:
from sklearn.metrics import mean_absolute_error
import numpy as np

E_p = np.array(df_predict[df_predict['dtype'] == 'energy']['y_orig'])/df_predict[df_predict['dtype'] == 'energy']['n']
E_o = np.array(df_orig[df_orig['dtype'] == 'energy']['y_orig'])/df_orig[df_orig['dtype'] == 'energy']['n']
print("MAE of training energy prediction is {} meV/atom".format(mean_absolute_error(E_o,E_p)*1000))

F_p = np.array(df_predict[df_predict['dtype'] == 'force']['y_orig'])/df_predict[df_predict['dtype'] == 'force']['n']
F_o = np.array(df_orig[df_orig['dtype'] == 'force']['y_orig'])/df_orig[df_orig['dtype'] == 'force']['n']
print("MAE of training force prediction is {} eV/Å".format(mean_absolute_error(F_o,F_p)))

MAE of training energy prediction is 3.0247720612186946 meV/atom
MAE of training force prediction is 0.13388747939814816 eV/Å


# Write and load fitted mtp with parameters files

In [ ]:
mtp_write = mtp.write_param()

In [11]:
mtp_loaded = MTPotential.from_config(filename='fitted.mtp', elements=["Ni", "Mo"])

# Lattice constants and eslastic constants
### Large error due to limited training data -- 10 structures

In [18]:
from pymatgen.core import Lattice

Ni = Structure.from_spacegroup(sg='Fm-3m', species=['Ni'], lattice=Lattice.cubic(3.51), coords=[[0, 0, 0]])
Mo = Structure.from_spacegroup(sg='Im-3m', species=['Mo'], lattice=Lattice.cubic(3.17), coords=[[0, 0, 0]])

In [19]:
from maml.apps.pes import LatticeConstant

lc_calculator = LatticeConstant(ff_settings=mtp_loaded)
a, b, c = lc_calculator.calculate([Ni])[0]
print('Ni', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

lc_calculator = LatticeConstant(ff_settings=mtp_loaded)
a, b, c = lc_calculator.calculate([Mo])[0]
print('Mo', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_serial
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_serial


Ni Lattice a: 4.24128207268093, Lattice b: 4.24128207268093, Lattice c: 4.24128207268093
Mo Lattice a: 3.74106964347794, Lattice b: 3.74106964347794, Lattice c: 3.74106964347794


In [20]:
from maml.apps.pes import ElasticConstant

Ni_ec_calculator = ElasticConstant(ff_settings=mtp_loaded)
Ni_C11, Ni_C12, Ni_C44, _ = Ni_ec_calculator.calculate([Ni])[0]
print('Ni', ' C11: ', Ni_C11, 'C12: ', Ni_C12, 'C44: ', Ni_C44)

Mo_ec_calculator = ElasticConstant(ff_settings=mtp_loaded)
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate([Mo])[0]
print('Mo', ' C11: ', Mo_C11, 'C12: ', Mo_C12, 'C44: ', Mo_C44)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_serial
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_serial


Ni  C11:  543.18248948548 C12:  506.8766660057 C44:  63.1820936905847
Mo  C11:  687.416829999536 C12:  848.638068500906 C44:  -117.680068651214


# Energy, force, stress prediction

In [8]:
from maml.apps.pes import EnergyForceStress

efs_calculator = EnergyForceStress(ff_settings=mtp_loaded)
energy, forces, stresses = efs_calculator.calculate([train_structures[0]])[0]

print('energy: {}'.format(energy))
print('forces: \n', forces)
print('stresses: ', stresses)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_intel_cpu_intelmpi


energy: -1022.29786524542
forces: 
 [[-1.80669452e-01  5.85713495e-01 -4.91113028e-02]
 [ 5.85236910e-01 -3.11719268e-01 -7.69291819e-01]
 [-4.12067711e-01 -2.51957173e-02  5.91975833e-01]
 [ 2.42062027e-01  4.82595209e-01 -4.24589110e-01]
 [-6.27320628e-01  9.73562484e-02 -1.52999562e+00]
 [ 5.23154452e-01  8.67986060e-02 -4.90606686e-01]
 [-7.71949477e-01  2.23083660e-01  3.56146825e-01]
 [-6.45754685e-02  1.22682423e-01  7.53082381e-01]
 [-1.32214890e+00 -4.46277203e-01  1.71445660e-01]
 [ 8.25921146e-01 -2.73233029e-01 -5.61798594e-01]
 [ 1.10181604e-01 -3.37781143e-02 -1.08024688e-01]
 [-8.35590493e-01  1.70547351e-01 -5.10978217e-01]
 [-6.84868741e-02  5.60219313e-02  5.75852435e-01]
 [-1.56871284e-01 -5.44826905e-01 -7.16374998e-02]
 [ 2.49752101e-01 -6.89926238e-01 -1.33197613e+00]
 [-5.90792743e-01 -5.10505650e-01 -4.97203873e-03]
 [-1.75215991e-01  2.85600825e-02 -1.54378059e-01]
 [-2.93483058e-01  1.30907176e-01 -6.35004914e-01]
 [-1.49449318e-03 -3.61541761e-01  1.24760421e